In [26]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [19]:
df = pd.read_csv('df/stories.csv')

In [20]:
df = df[(df['media_id'] <= 33) & (df['media_id'] > 0)].copy()
df = df[df['link'] == df['link']].copy()
df = df[df['link'] != '#'].copy()

In [21]:
# Sorry for this
def getHeading(_media_id, _link):
    url_parts = _link.split('/')

    # tg, vqh, lm        
    if _media_id in (19, 20, 25):
        return url_parts[1]
            
    # le temps
    elif _media_id == 21:
        if url_parts[1] == 'node':
            return 'longform'
        else:
            return url_parts[1]
    
    # le monde
    elif _media_id == 22:
        return url_parts[1]
    
    # rts info
    elif _media_id == 23:
        return url_parts[2]
    
    # mediapart
    elif _media_id == 26:
        return url_parts[2]
    
    # Libé
    elif _media_id == 28:
        if url_parts[1] in ['france', 'politiques']:
            return 'national'
        elif url_parts[1] == 'planete':
            return 'international'
        else:
            return url_parts[1]
    
    # 20 minutes
    elif _media_id == 24:
        if url_parts[2] != 'news':
            return url_parts[2]
            
        else:
            return url_parts[3]
        
    # le figaro (27): 3
    elif _media_id == 27:
        
        return url_parts[3]
    
    # La Côte, Le Nouvelliste, La Liberté
    elif _media_id in (29, 30, 31, 32):
        if url_parts[2] in ('valais', 'regions', 'canton'):
            return 'région'
        else:
            return url_parts[2]

    # Le Courrier    
    elif _media_id == 33:
        return ''
    
    else:
        print('Media', _media_id, 'not in list missing')
        return 'missing' + str(_media_id)

In [22]:
df['heading'] = ''
for i, row in df.iterrows():
    if row['media_id'] != 33: # Le Courrier: manuel
        df.at[i, 'heading'] = getHeading(row['media_id'], row['link'])
    else:
        # These headings get fixed right below
        print('C', end='')

CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC

In [24]:
# Add heading for Le Courrier
print("We’ll get {} missing headings for “Le Courrier”".format( df[df['media_id'] == 33].shape[0]) )

We’ll get 60 missing headings for “Le Courrier”


In [27]:
for i, row in df[df['media_id'] == 33].iterrows():
    print(row['absolute_url'])
    ua_headers = {'User-Agent': 'Quel-media-bot (+https://www.quel-media.com/about.html#robot)'}
    response = requests.get(row['absolute_url'], headers=ua_headers)
    if not response:
        print('Error loading', row['absolute_url'])
    elif response.status_code != 200:
        print('Error {} when loading {}'.format( response.status_code, row['url'] ) )
    doc = BeautifulSoup(response.content, 'html.parser')    
    tags = doc.select('.c-Article-theme .c-Article-breadcrumb-tagsLink')
    print(tags)
    if len(tags) > 0:
        tag = tags[0]
        print(tag.string, tag.get('href'))
        df.at[i, 'heading'] = tag.string
    else:
        print('No tag found')
    print()

https://lecourrier.ch/2018/05/27/au-mexique-la-resistance-au-fracking-sorganise/ 
[<a class="c-Article-breadcrumb-tagsLink" href="/theme/solidarite/">Solidarité</a>]
Solidarité /theme/solidarite/

https://lecourrier.ch/2018/05/27/une-initiative-zero-pub-au-rabais/ 
[<a class="c-Article-breadcrumb-tagsLink" href="/theme/geneve/">Genève</a>]
Genève /theme/geneve/

https://lecourrier.ch/2018/05/28/inclure-le-citoyen-dans-la-decision/ 
[<a class="c-Article-breadcrumb-tagsLink" href="/rubrique/international/">International</a>]
International /rubrique/international/

https://lecourrier.ch/2018/05/29/budget-le-mcg-lache-la-droite/ 
[<a class="c-Article-breadcrumb-tagsLink" href="/theme/geneve/">Genève</a>]
Genève /theme/geneve/

https://lecourrier.ch/2018/05/28/les-tpg-accuses-de-censure-politique/ 
[<a class="c-Article-breadcrumb-tagsLink" href="/theme/geneve/">Genève</a>]
Genève /theme/geneve/

https://lecourrier.ch/2018/05/29/lausanne-senflamme-sur-le-deal/ 
[<a class="c-Article-breadcrum

[<a class="c-Article-breadcrumb-tagsLink" href="/theme/egalite/">Égalité</a>]
Égalité /theme/egalite/

https://lecourrier.ch/2018/06/18/super-collaboratif/ 
[<a class="c-Article-breadcrumb-tagsLink" href="/theme/alternatives/">Alternatives</a>]
Alternatives /theme/alternatives/

https://lecourrier.ch/2018/06/18/mexique-le-pays-des-disparus/ 
[<a class="c-Article-breadcrumb-tagsLink" href="/theme/solidarite/">Solidarité</a>]
Solidarité /theme/solidarite/

https://lecourrier.ch/2018/06/18/voltigeuses-dans-la-lumiere/ 
[<a class="c-Article-breadcrumb-tagsLink" href="/theme/scene/">Scène</a>]
Scène /theme/scene/

https://lecourrier.ch/2018/06/19/davantage-de-gym-moins-dallemand/ 
[<a class="c-Article-breadcrumb-tagsLink" href="/theme/geneve/">Genève</a>]
Genève /theme/geneve/

https://lecourrier.ch/2018/06/20/en-quete-douvertures/ 
[<a class="c-Article-breadcrumb-tagsLink" href="/theme/religions/">Religions</a>]
Religions /theme/religions/

https://lecourrier.ch/2018/06/20/grand-saconnex-a

In [35]:
df.to_csv('df/stories_with_heading.csv', index=False)

In [28]:
df[df['media_id'] == 33]['heading'].value_counts()

Genève           21
International     8
Vaud              8
Solidarité        6
                  3
Égalité           2
Suisse            2
Neuchâtel         2
Écologie          1
Livres            1
Économie          1
Alternatives      1
Scène             1
Religions         1
Régions           1
Musique           1
Name: heading, dtype: int64

In [30]:
len(df[(df['media_id'] == 33) & (df['heading'].isin(['International', 'Solidarité']))])

14

In [32]:
len(df[df['media_id'] == 33])

60

In [33]:
14 / 60

0.23333333333333334